<a href="https://colab.research.google.com/github/Dorineoloo/Chat-Bot/blob/main/Copy_of_chatbot_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Things we need for NLP
import nltk #Natural Language Processing Tool Kit
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Things we need for Tensorflow
import tensorflow as tf
import numpy as np
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
! pip install tflearn

In [ ]:
import tflearn

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents":[\r\n    {\r\n        "tag": "greeting",\r\n        "patterns": ["Hi", "Hey", "Hello", "How are you?", "Good day", "Is anyone there?"],\r\n        "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help you?"],\r\n        "context_set": ""\r\n    },\r\n    {\r\n        "tag": "goodbye",\r\n        "patterns": ["Bye", "Goodbye", "See you later"],\r\n        "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\r\n    },\r\n    {\r\n        "tag": "thanks",\r\n        "patterns": ["Thanks", "Thank you", "That\'s helpful"],\r\n        "responses": ["Happy to help", "You\'re welcome always", "Anytime", "My pleasure"]   \r\n    },\r\n    {\r\n        "tag": "hours",\r\n        "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\r\n        "responses": ["We\'re open every week day 8am-5pm", "Our hours are 8am-5pm for week days"]\r\n    },\r\

In [ ]:
# Import our chatbot intents file
import json
with open('intents.json') as json_data:
  intents = json.load(json_data)

In [ ]:
from nltk.corpus.reader import documents
words = []
classes = []
documents = []
ignore_words = ['?']

# looping through each sentence in our intents patterns
for intent in intents['intents']:
  for pattern in intent['patterns']:
    # tokenization
    w = nltk.word_tokenize(pattern)
    # add to our words list
    words.extend(w)
    # add to documents in our corpus
    documents.append((w, intent['tag']))
    # add to our classes list
    if intent['tag'] not in classes:
        classes.append(intent['tag'])


# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

43 documents
15 classes ['challenges when connecting to lockdown browser exams', 'connection server blocked', 'contact Kabuo', 'email', 'goodbye', 'greeting', 'hours', 'installing lockdown browser', 'launching lockdown browser', 'moodlelogin', 'respondus', 'student portal', 'thanks', 'video connection timed out', 'zoom']
89 unique stemmed words ["'s", ',', 'a', 'abl', 'access', 'am', 'an', 'anyon', 'ar', 'block', 'brows', 'bye', 'can', 'cant', 'connect', 'continu', 'day', 'do', 'down', 'email', 'er', 'exam', 'get', 'good', 'goodby', 'hang', 'has', 'hav', 'hello', 'help', 'hey', 'hi', 'hour', 'how', 'i', 'in', 'inform', 'instal', 'into', 'is', 'join', 'lat', 'launch', 'link', 'lock', 'lockdown', 'lockdown-browser', 'log', 'login', 'meet', 'moodl', 'my', 'navig', 'new', 'not', 'op', 'or', 'out', 'pag', 'platform', 'port', 'problem', 'respond', 'see', 'send', 'serv', 'start', 'stuck', 'stud', 'supply', 'thank', 'that', 'the', 'ther', 'though', 'tim', 'to', 'un', 'upd', 'us', 'video', 'wha

In [ ]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
  # initialize bag of words
  bag = []
  # create list for tokenized words for the patterns
  pattern_words = doc[0]
  # stem each word 
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  # create our bag of words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  
  # output is a '0' for each tag and '1' for current tag
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1

  training.append([bag, output_row])

# shuffle our features to numpy array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
# save all of your data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open("training_data", "wb"))

In [ ]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, batch_size=8, show_metric=True, n_epoch=1000)
model.save('model.chatbot')

Training Step: 5999  | total loss: 0.00079 | time: 0.027s
| Adam | epoch: 1000 | loss: 0.00079 - acc: 1.0000 -- iter: 40/43
Training Step: 6000  | total loss: 0.00081 | time: 0.035s
| Adam | epoch: 1000 | loss: 0.00081 - acc: 1.0000 -- iter: 43/43
--


In [ ]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [ ]:
p = bow("is your Kabuo open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
['challenges when connecting to lockdown browser exams', 'connection server blocked', 'contact Kabuo', 'email', 'goodbye', 'greeting', 'hours', 'installing lockdown browser', 'launching lockdown browser', 'moodlelogin', 'respondus', 'student portal', 'thanks', 'video connection timed out', 'zoom']


In [ ]:
# load our saved model
model.load('model.chatbot')

In [ ]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='130', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)


In [ ]:
classify('is KABUO open today?')

[('goodbye', 0.49331892), ('greeting', 0.32445595)]

In [ ]:
response("Hi")

Hello, thanks for visiting


In [ ]:
response("My zoom!!")

LockDown Browser is the “gold standard” for securing online exams in classrooms or proctored environments.


In [ ]:
response("What is Lockdown")

LockDown Browser is the “gold standard” for securing online exams in classrooms or proctored environments.


In [ ]:
response("zoom")

Hi there, how can I help you?


In [ ]:
response("I cannot login to moodle")

Please reset your password and check your school email


In [ ]:
response("zoom")

Hi there, how can I help you?


In [ ]:
response("My video is hanging")

Change your internet connection


In [ ]:
response("My video is hanging")

Check your internet connection 


In [ ]:
response("I cannot login to moodle")

Please reset your password and check your school email


In [ ]:
response("cant login")

credentials registration number


In [ ]:
response("Hi")

Good to see you again


In [ ]:
response("connection has timed out")

Check your internet connection 


In [ ]:
response("connection has timed out")

Change your internet connection


In [ ]:
response("connection has timed out")

Change your internet connection


In [ ]:
response("connection has timed out")

Check your internet connection 


In [ ]:
response("my lockdown browser page is stuck")

Restart your lock down browser


In [ ]:
response("my lockdown browser page is stuck")

Restart your computer 


In [ ]:
response("moodle is stuck")

Please reset your password and check your school email


In [ ]:
response("cant access moodle cat")

share phone number


In [ ]:
response("new version of respondus lockdown")

LockDown Browser® is a custom browser that locks down the testing environment within a learning management system.


In [ ]:
response("new version of respondus lockdown")

LockDown Browser® is a custom browser that locks down the testing environment within a learning management system.


In [ ]:
response("moodle update")

share phone number
